Leak Sites Indexing Tool
========================

Dieses Jupyter-Notebook scraped verschiedene Ressourcen, die Leak Sites sammeln, und vergleicht ihre Inhalte.

Voraussetzungen
---------------

Um die Verfügbarkeit von Leak Sites zu prüfen, wird ein Tor-Proxy benötigt.

Unter Linux kann dafür das Tor-Programm installiert und mit `systemctl start tor` werden.
Dann läuft auf dem Port 9050 ein lokaler Tor-Socks-Proxy, über den dieses Notebook Onion-Links aufrufen kann.
Wenn der Tor-Proxy anders zur Verfügung gestellt wird, kann hier auch eine andere Proxy-Adresse konfiguriert werden.

In [ ]:
tor_proxy = 'socks5h://localhost:9050'

Proxy-Test
----------

Die Ausgabe der nächsten Zelle sollte "✅ You are using Tor!" sein, ansonsten können Onion-Seiten nicht aufgerufen werden.
Dann werden alle Leak Sites fälschlicherweise als Offline angezeigt.

In [ ]:
from requests import session, get

tor = session()
tor.proxies.update({'http': tor_proxy, 'https': tor_proxy})

resp = tor.get('https://check.torproject.org', timeout=60)
if 'Congratulations. This browser is configured to use Tor.' in resp.text:
    print('✅ You are using Tor!')
else:
    print('❌ You are NOT using Tor!')

# [fastfire/deepdarkCTI](https://github.com/fastfire/deepdarkCTI/blob/main/ransomware_gang.md)

In [ ]:
import csv
import re
import pandas as pd

deepdarkcti_md = get('https://raw.githubusercontent.com/fastfire/deepdarkCTI/refs/heads/main/ransomware_gang.md', timeout=60).text
lines = deepdarkcti_md.splitlines()
lines.pop(1) # Header-Endzeile entfernen
lines = [line.strip('|') for line in lines] # '|' Am Anfang und Ende der Zeile entfernen

deepdarkcti_reader = csv.DictReader(lines, delimiter='|')

deepdarkcti = { 'Group': [], 'Link': [], 'Available': [], 'AvailableLabel': [] }
for row in deepdarkcti_reader:
    parsed_name = re.search(r"\[([^]]*)]\(([^)]*)\)", row['Name'])
    if parsed_name is not None:
        if parsed_name.group(1) in [
            'Ransomfeed',
            'eCrime Services',
            'RANSOM DB',
            'RANSOMWARE GROUP SITES (list)',
            'RANSOMWARE GROUPS MONITORING TOOL',
            'RansomChats'
        ]:
            continue
        deepdarkcti['Group'].append(parsed_name.group(1))
        deepdarkcti['Link'].append(parsed_name.group(2))
        deepdarkcti['AvailableLabel'].append('ONLINE' in row['Status'])
        print(f"Checking {parsed_name.group(1)} ({parsed_name.group(2)})")
        try:
            tor.get(parsed_name.group(2), timeout=60)
            deepdarkcti['Available'].append(True)
        except:
            deepdarkcti['Available'].append(False)
        
deepdarkcti = pd.DataFrame(deepdarkcti)
deepdarkcti

# [u/DrinkMoreCodeMore](https://www.reddit.com/r/Malware/comments/1bpcrdw/list_of_ransomware_groups_and_their_pr_pages_2024)

In [ ]:
import bs4

reddit2024_soup = bs4.BeautifulSoup(get('https://www.reddit.com/r/Malware/comments/1bpcrdw/list_of_ransomware_groups_and_their_pr_pages_2024/', timeout=60).text)
reddit2024_soup = reddit2024_soup.css.select('shreddit-post [slot=text-body] > div > div > p')

reddit2024 = { 'Group': [], 'Link': [], 'Available': [] }
for p in reddit2024_soup:
    if p.find('a') is None:
        continue
    parsed_text = re.search(r"([^]]*) - ([^)]*)", p.text)
    if parsed_text is not None:
        reddit2024['Group'].append(parsed_text.group(1).strip())
        reddit2024['Link'].append(parsed_text.group(2).strip())
        print(f"Checking {parsed_text.group(1).strip()} ({parsed_text.group(2).strip()})")
        try:
            tor.get(parsed_text.group(2).strip(), timeout=60)
            reddit2024['Available'].append(True)
        except:
            reddit2024['Available'].append(False)
            
reddit2024 = pd.DataFrame(reddit2024)
reddit2024

# [Onion-Seite](http://ransomwr3tsydeii4q43vazm7wofla5ujdajquitomtd47cxjtfgwyyd.onion)

In [ ]:
import bs4

darknet_soup = bs4.BeautifulSoup(tor.get('http://ransomwr3tsydeii4q43vazm7wofla5ujdajquitomtd47cxjtfgwyyd.onion/', timeout=60).text)
darknet_soup = darknet_soup.css.select_one('.table .table-body')
darknet_soup = darknet_soup.css.select('.table-row')

darknet = { 'Group': [], 'Link': [], 'Available': [] }
for row in darknet_soup:
    name = row.css.select_one('.table-cell')
    links = row.css.select('a')
    for link in links:
        darknet['Group'].append(name.text)
        darknet['Link'].append(link.attrs['href'])
        print(f"Checking {name.text} ({link.attrs['href']})")
        try:
            tor.get(link.attrs['href'], timeout=60)
            darknet['Available'].append(True)
        except:
            darknet['Available'].append(False)

darknet = pd.DataFrame(darknet)
darknet

# [Ransomwatch](https://ransomwatch.telemetry.ltd)

In [ ]:
import csv

ransomwatch_md = get('https://ransomwatch.telemetry.ltd/INDEX.md', timeout=60).text
lines = ransomwatch_md.splitlines()[2:-1]
lines.pop(1) # Header-Endzeile entfernen
lines = [line.strip('|') for line in lines] # '|' Am Anfang und Ende der Zeile entfernen

ransomwatch_reader = csv.DictReader(lines, delimiter='|')

ransomwatch = { 'Group': [], 'Available': [], 'Link': [], 'AvailableLabel': [] }
for row in ransomwatch_reader:
    if len(row[' group '].strip()) == 0:
        continue
    parsed_name = re.search(r"\[([^]]*)]\(([^)]*)\)", row[' group '])
    ransomwatch['Group'].append(parsed_name.group(1))
    ransomwatch['Link'].append(row[' location '])
    if row[' status '] is not None:
        ransomwatch['AvailableLabel'].append('🟢' in row[' status '])
    else:
        ransomwatch['AvailableLabel'].append(False)
    print(f"Checking {parsed_name.group(1)} ({row[' location ']})")
    try:
        tor.get(row[' location '], timeout=60)
        ransomwatch['Available'].append(True)
    except:
        ransomwatch['Available'].append(False)
        
    
ransomwatch = pd.DataFrame(ransomwatch)
ransomwatch['Available'] = ransomwatch['Available'].astype('bool')
ransomwatch

# [Ransomfind](https://ransomfind.io)

In [ ]:
import csv

ransomfind_md = get('https://ransomfind.io/INDEX.md', timeout=60).text
lines = ransomfind_md.splitlines()[2:-1]
lines.pop(1) # Header-Endzeile entfernen
lines = [line.strip('|') for line in lines] # '|' Am Anfang und Ende der Zeile entfernen

ransomfind_reader = csv.DictReader(lines, delimiter='|')

ransomfind = { 'Group': [], 'Available': [], 'Link': [], 'AvailableLabel': [] }
for row in ransomfind_reader:
    parsed_name = re.search(r"\[([^]]*)]\(([^)]*)\)", row[' group '])
    ransomfind['Group'].append(parsed_name.group(1))
    ransomfind['Link'].append(row[' location '].strip())
    ransomfind['AvailableLabel'].append('🟢' in row[' status '])
    print(f"Checking {parsed_name.group(1)} ({row[' location '].strip()})")
    try:
        tor.get(row[' location '].strip(), timeout=60)
        ransomfind['Available'].append(True)
    except:
        ransomfind['Available'].append(False)

ransomfind = pd.DataFrame(ransomfind)
ransomfind['Available'] = ransomfind['Available'].astype('bool')
ransomfind

# [Ransomlook](https://www.ransomlook.io)

In [ ]:
import bs4

ransomlook_soup = bs4.BeautifulSoup(get('https://www.ransomlook.io/groups', timeout=60).text)

ransomlook = { 'Group': [], 'Available': [], 'Link': [], 'AvailableLabel': [] }
for header in ransomlook_soup.css.select('h3'):
    following = header.find_next('div')
    if following is not None:
        links = following.css.select_one('tbody')
        for link in links.css.select('tr'):
            ransomlook['Group'].append(header.text)
            ransomlook['Link'].append(link.css.select_one('td:nth-child(4)').text)
            ransomlook['AvailableLabel'].append('⬆️' in link.css.select_one('td:nth-child(2)').text)
            print(f"Checking {header.text} ({link.css.select_one('td:nth-child(4)').text})")
            try:
                tor.get(link.css.select_one('td:nth-child(4)').text, timeout=60)
                ransomlook['Available'].append(True)
            except:
                ransomlook['Available'].append(False)

ransomlook = pd.DataFrame(ransomlook)
ransomlook['Available'] = ransomlook['Available'].astype('bool')
ransomlook


# [Ransomfeed](https://ransomfeed.it)

In [ ]:
import bs4

ransomfeed = { 'Group': [], 'Link': [], 'Available': [], 'AvailableLabel': [] }
ransomfeed_soup = bs4.BeautifulSoup(get('https://ransomfeed.it/stats.php?page=groups-stats', timeout=60).text)
ransomfeed_soup = ransomfeed_soup.find('tbody')
ransomfeed_soup = ransomfeed_soup.css.select('tr')

for row in ransomfeed_soup:
    name = row.css.select_one('a')
    group_soup = bs4.BeautifulSoup(get(f"https://ransomfeed.it{name.attrs['href']}").text)
    card = group_soup.css.select_one('.card:nth-child(3)')
    if card is None:
        continue
    for ro in card.css.select('tr:not(:first-child)'):
        ransomfeed['Group'].append(name.text)
        ransomfeed['Link'].append(ro.css.select_one('td:nth-child(1)').text)
        ransomfeed['AvailableLabel'].append('🟢' in ro.css.select_one('td:nth-child(3)').text)
        print(f"Checking {name.text} ({ro.css.select_one('td:nth-child(1)').text})")
        try:
            tor.get(ro.css.select_one('td:nth-child(1)').text, timeout=60)
            ransomfeed['Available'].append(True)
        except:
            ransomfeed['Available'].append(False)

ransomfeed = pd.DataFrame(ransomfeed)
ransomfeed

# [ransomwarelive](https://www.ransomware.live)

In [ ]:
import bs4

ransomwarelive = { 'Group': [], 'Link': [], 'Available': [], 'AvailableLabel': [] }
ransomwarelive_soup = bs4.BeautifulSoup(get('https://www.ransomware.live/groups/', timeout=60).text)
ransomwarelive_soup = ransomwarelive_soup.find('tbody')
ransomwarelive_soup = ransomwarelive_soup.css.select('tr')

for row in ransomwarelive_soup:
    name = row.css.select_one('a')
    group_soup = bs4.BeautifulSoup(get(f"https://www.ransomware.live{name.attrs['href']}").text, timeout=60)
    card = group_soup.css.select_one('tbody')
    if card is None:
        continue
    for ro in card.css.select('tr'):
        ransomwarelive['Group'].append(name.text)
        ransomwarelive['Link'].append(ro.css.select_one('td:nth-child(4)').text)
        ransomwarelive['AvailableLabel'].append('🟢' in ro.css.select_one('td:nth-child(2)').text)
        print(f"Checking {name.text} ({ro.css.select_one('td:nth-child(4)').text})")
        try:
            tor.get(ro.css.select_one('td:nth-child(4)').text, timeout=60)
            ransomwarelive['Available'].append(True)
        except:
            ransomwarelive['Available'].append(False)

ransomwarelive = pd.DataFrame(ransomfeed)
ransomwarelive



# TODO

Wie viel Prozent falsch online angezeigt?
Wie viel Prozent der Posts kriegen wir?

In [ ]:
from script.normalizer import normalize

normalize(deepdarkcti, reddit2024, darknet, ransomwatch, ransomfind, ransomlook, ransomwarelive, ransomfeed)

all_groups = pd.concat([
    deepdarkcti,
    reddit2024,
    darknet,
    ransomwatch,
    ransomfind,
    ransomlook,
    ransomwarelive,
    ransomfeed
])

In [ ]:
def get_tracked_status(only_active, indexer_dict: dict):
    tracking_status = { 'Group': [] }
    for i in indexer_dict.keys():
        tracking_status[i] = []
    
    concatted = all_groups
    if only_active:
        concatted = concatted[concatted['Available']]
    total = concatted['Group'].unique()

    for i in indexer_dict.keys():
        indexer = indexer_dict[i]
        if only_active:
            indexer = indexer[indexer['Available']]
        for group in total:
            tracking_status[i].append((indexer['Group'] == group).any())

    for group in total:
        tracking_status['Group'].append(group)
    
    return pd.DataFrame(tracking_status)

indexers = {
    'deepdarkcti': deepdarkcti,
    'reddit2024': reddit2024,
    'darknet': darknet,
    'ransomwatch': ransomwatch,
    'ransomfind': ransomfind,
    'ransomlook': ransomlook,
    'ransomwarelive': ransomwarelive,
    'ransomfeed': ransomfeed
}

tracking_status_all = get_tracked_status(False, indexers)

groups_all = tracking_status_all['Group'].count()
relative_all = tracking_status_all.sum(numeric_only=True).transform(lambda x: x * 100 / groups_all)

ax_all = relative_all.plot.barh(xlim=[0, 100])
ax_all.bar_label(ax_all.containers[0], fmt="%.3g %%")

print(groups_all)


In [ ]:
import json
import pandas as pd

keywords = [
    "medical",
    "hospital",
    "clinic",
    "insur",
    "dent",
    "patient",
    "recover",
    "doctor",
    "nurs",
    "health",
    "facilit",
    "care",
    "lab",
    "pediat",
    "rehab",
    "krank",
    "mediz",
    "gesund",
]

with open("data/posts.json", "r") as f:
    posts = json.load(f)

df = pd.json_normalize(posts)

filtered = df['post_title'].apply(lambda d: any(keyword in d.lower() for keyword in keywords))

df = df[filtered]

df['discovered'] = pd.to_datetime(df['discovered'])
df['discovered'] = df['discovered'].dt.to_period('M')

df

In [ ]:
from pandas import DataFrame, to_datetime

results_all = { 'Title': [], 'Group': [], 'Date': [] }
results_medical = { 'Title': [], 'Group': [], 'Date': [] }
for post in posts:
    title = post['post_title'].lower()
    results_all['Title'].append(post['post_title'])
    results_all['Group'].append(post['group_name'])
    results_all['Date'].append(post['discovered'])
    for keyword in keywords:
        if keyword in title:
            results_medical['Title'].append(post['post_title'])
            results_medical['Group'].append(post['group_name'])
            results_medical['Date'].append(post['discovered'])
            break

df_all = DataFrame(results_all)

df_all['Date'] = to_datetime(df_all['Date'])
df_all['Month'] = df_all['Date'].dt.to_period('M')
print(len(results_all['Title']))

data_all = df_all['Month'].value_counts()

df_medical = DataFrame(results_medical)

df_medical['Date'] = to_datetime(df_medical['Date'])
df_medical['Month'] = df_medical['Date'].dt.to_period('M')
print(len(results_medical['Title']))


data_medical = df_medical['Month'].value_counts()

all = DataFrame({
    'all': data_all,
    'medical': data_medical,
})
all['part'] = all['medical'] / all['all']
all